In [3]:
#ON IMPORTE PLEIN DE PACKAGES
import os
import tweepy as tw
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import nltk
from nltk.corpus import stopwords
import re
import networkx

from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer

import six

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

In [6]:
#CODES D'ACCES DE NOTRE API
consumer_key= 'vKEVNk4bU3SLIyX1St4VFAHxx'
consumer_secret= 'Vph1hkE8qi3m6acec8YXSjuYLfNhWJZzr9AEx9N20L8uohl7Km'
access_token= '1454759642228215809-fVDHUEShqBaH32XaNmunO5rKulOEQQ'
access_token_secret= 'GFytsNLAQUXcGdAc6mldJ8bM2ruVNtDkCZrPMg3weY6ZM'

#ON SE CONNECTE A L'API 
auth = tw.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

#ON CREE L'OBJET API
api = tw.API(auth, wait_on_rate_limit=True)

#ON DEFINIT LA DATE D'AUJOURD'HUI
today = datetime.date.today()
yesterday= today - datetime.timedelta(days=1)

In [63]:
#TEST 1 : 
#ON COLLECTE DES TWEETS FR :
nbr=1000
search_term = "#bitcoin"

tweets = tw.Cursor(api.search_tweets,
                   q=search_term,
                   lang="en",
                   since_id="2021-11-05",result_type = "popular").items(nbr)

all_tweets = [tweet.text for tweet in tweets]

all_tweets[:50]

['Using a watch to pay with #Bitcoin in El Salvador 🙌\n\nNo phone, no problem.\n\n https://t.co/addl3urkeh',
 '#Bitcoin is hope for #Turkey. The currency has lost a third of its value since March and declined for nine consecut… https://t.co/IaWloYUAvL',
 'Today I setup a #BlockstreamSatellite Base Station at the @LaGeoSV Berlin geothermal plant, where the Volcano… https://t.co/7RENWycyGW',
 "There's now a #Bitcoin satellite at El Salvador's volcano mine. \n\nEarth's energy powering interstellar money 💫 https://t.co/xga5OSBcCT",
 'BREAKING – Banking giant Fidelity approved to launch institutional #Bitcoin services in Canada https://t.co/v4fyfJyI4O',
 'This is the truth about #crypto, #Bitcoin, and Congress.\n \nBut Congress is trying to regulate it anyway. https://t.co/Yxj5tXr4pE',
 'Someone actually tried to steal this #Bitcoin ATM to get the coins out 😂 https://t.co/Jfpql2gP3e',
 'Take crypto-currency or #bitcoin for example. \n\nIt is important that all democratic nations work togeth

In [64]:
#FONCTION POUR ENLEVER URL D'UN TWEET
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """
    return " ".join(re.sub("([^j0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [65]:
#ON REPREND LA LISTE EN ENLEVANT LES URLs, LES HASHTAGS, LES @, LES _ ET LES MAJUSCULES
all_tweets_no_urls = [remove_url(tweet).replace("#", "").replace("_", " ").replace("@", " ").lower() for tweet in all_tweets]
all_tweets_no_urls[:50]

['using a watch to pay with bitcoin in el salvador no phone no problem',
 'bitcoin is hope for turkey the currency has lost a third of its value since march and declined for nine consecut',
 'today i setup a blockstreamsatellite base station at the lageosv berlin geothermal plant where the volcano',
 'theres now a bitcoin satellite at el salvadors volcano mine earths energy powering interstellar money',
 'breaking banking giant fidelity approved to launch institutional bitcoin services in canada',
 'this is the truth about crypto bitcoin and congress but congress is trying to regulate it anyway',
 'someone actually tried to steal this bitcoin atm to get the coins out',
 'take cryptocurrency or bitcoin for example it is important that all democratic nations work together on this a',
 'looking at future bitcoin volcanoes el salvador is spectacular',
 'is chivopets being built too fast bitcoin',
 'bitcoin is powered by hope',
 'enjoy the ride bitcoin',
 '40 of all dollars in circulation h

In [66]:
#PROBLEME 1 : LES ACCENTS DISPARAISSENT
#SOLUTION 1: ON TRAVAILLE AVEC LES TWEETS ANGLAIS, SANS ACCENTS
#SOLUTION 2: ON TROUVE DANS LE RETURN DE LA FONCTION REMOVE_URL LA COMMANDE QUI ENLEVE LES LETTRES ACCENTUEES
#J'ai regardé la documentation de .sub, cherché sur internet, c'est long et ça n'a pas aboutit

In [67]:
#TEST 2 : AVEC GOOGLE NLP PACKAGE
#ON IMPORTE LES PACKAGES
from google.cloud import language_v1
from google.cloud.language_v1 import types
import os

In [68]:
#ON SE CONNECTE A GOOGLE CLOUD 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\Asus\Desktop\M2 S1\Big-Data\coherent-elf-332016-1c937294d37f.json"
client = language_v1.LanguageServiceClient()

In [69]:
for tweet in all_tweets_no_urls[:20]:
    document = types.Document(content=tweet,type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment_score = client.analyze_sentiment(document=document).document_sentiment.score
    print(tweet)
    print(sentiment_score)

using a watch to pay with bitcoin in el salvador no phone no problem
-0.10000000149011612
bitcoin is hope for turkey the currency has lost a third of its value since march and declined for nine consecut
-0.699999988079071
today i setup a blockstreamsatellite base station at the lageosv berlin geothermal plant where the volcano
0.0
theres now a bitcoin satellite at el salvadors volcano mine earths energy powering interstellar money
0.0
breaking banking giant fidelity approved to launch institutional bitcoin services in canada
0.20000000298023224
this is the truth about crypto bitcoin and congress but congress is trying to regulate it anyway
-0.30000001192092896
someone actually tried to steal this bitcoin atm to get the coins out
-0.699999988079071
take cryptocurrency or bitcoin for example it is important that all democratic nations work together on this a
0.0
looking at future bitcoin volcanoes el salvador is spectacular
0.8999999761581421
is chivopets being built too fast bitcoin
-0.

In [74]:
score=0
i=0
for tweet in all_tweets_no_urls[:200]:
    document = types.Document(content=tweet,type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment_score = client.analyze_sentiment(document=document).document_sentiment.score
    score+=sentiment_score
    #print(score)
    i+=1
    #print(i)
score_final=score/200
print('Opinion générale sur Bitcoin',score_final)

Opinion générale sur Bitcoin: -0.006000000946223736


In [1]:
#PARTIE API BITMEX/HITBTC

In [3]:
#NOS IDENTIFIANTS API
API_KEY='vkycdshFDcepfC64ESAmMfRB'
API_SECRET='RmUYKgXzKYS3WO2fjWe1McnmPk4CxtLB8BsI260w4ZpZHWis'

In [5]:
#ON SE CONNECTE A L'API BITMEX
import bitmex
import time, requests, json
bitmex_api_key = API_KEY
bitmex_api_secret = API_SECRET
client = bitmex.bitmex(test = True, api_key=bitmex_api_key, api_secret=bitmex_api_secret) 

C:\Users\Asus\anaconda3\lib\site-packages\swagger_spec_validator\validator20.py:48: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  warnings.warn(


In [ ]:
#ON CHOISIT LA CRYPTO ET LE TYPE DE L'ORDRE
symbol = 'XBTUSD'
ordType = 'Market'

In [ ]:
if score_final>0:#Si score_final > 0 : ON ACHETE DU BITCOIN POUR 100$
    orderQty_Buy = 100 #VALEUR POSITIVE=ACHAT
    client.Order.Order_new(symbol=symbol, ordType=ordType, orderQty=orderQty_Buy).result() # ACHAT
else:#Si score_final < 0 : ON VEND DU BITCOIN POUR 100$
    orderQty_Sell = -100 #VALEUR NEGATIVE=VENTE
    client.Order.Order_new(symbol=symbol, ordType=ordType, orderQty=orderQty_Sell).result() # VENTE

In [ ]:
#ON BOUCLE SUR score_final POUR IDENTIFIER QUAND ON FERME NOTRE POSITION
if score_final>0:
    score=0
    i=0
    N=200
    for tweet in all_tweets_no_urls[:N]:
        document = types.Document(content=tweet,type_=language_v1.Document.Type.PLAIN_TEXT)
        sentiment_score = client.analyze_sentiment(document=document).document_sentiment.score
        score+=sentiment_score
        #print(score)
        i+=1
        #print(i)
        score_final2=score/N
    if score_final2<score_final:
        client.Order.Order_closePosition(symbol=symbol).result() #ON FERME TOUTES NOS POSITIONS
#PAREIL DANS L'AUTRE CAS
if score_final<=0:
    score=0
    i=0
    N=200
    for tweet in all_tweets_no_urls[:N]:
        document = types.Document(content=tweet,type_=language_v1.Document.Type.PLAIN_TEXT)
        sentiment_score = client.analyze_sentiment(document=document).document_sentiment.score
        score+=sentiment_score
        #print(score)
        i+=1
        #print(i)
        score_final2=score/N
    if score_final2>score_final:
        client.Order.Order_closePosition(symbol=symbol).result() #ON FERME TOUTES NOS POSITIONS